# Who: Andrei Olariu
## Stone Soup Technology
<br/>
<br/>


# What: Matching Journalists with Domain Experts
## Text Classification with BERT and Gradient Boosting Trees from Idea to Production

- intro on me and StoneSoup

- describing the client and his problem

- baseline approach

- Machine Learning approach

- discussion

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/sst.png" width="800vmin" style="padding: 0 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/andrei.jpg" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

that's me during the pandemic

phd in natural language processing

3 top 10 finishes in kaggle contests

at sst, involved in backend, api and ml work

now you have more context on me and the company I work at

unfortunately, I don't know a lot about you, so I had to prepare this workshop making some assumptions

if you have experience with something and you find some things trivial, then just breath in, breath out, enjoy the moment and bask in your awesomeness

don't wait till the end for questions

# Assumptions

- no/little knowledge of Python
- no/little experience building and deploying Machine Learning models

# Feel free to interrupt me and ask questions

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/guidedpr.png" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

We have a database of members, with a short description for each

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/brad.png" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

We have new requests coming in from journalists and we want to identify the best matching members

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/request1.png" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

What do you think?

How can we approach this problem?

We need a baseline approach. It needs to be:

- easy to implement

- easy to explain/understand/debug

- predictable

- independent of training data

Cmon, let's hear some ideas

actively thinking about a solution will stimulate the creation of new neural paths inside your brain, making you better problem solvers

i swear I'm not moving forward until I get another idea

Our baseline approach:

- index the members' descriptions using a search engine (Solr)

- given a request, send it to the search engine as the query and get the highest ranking members

- **have a moderator review and correct the matches**

- send updated matches to the journalist

notice the third step (hard not to)

we don't trust the current approach - and it's right so, we haven't evaluated it

we are also building a manually annotated dataset to train a ML model

In [3]:
import pickle

with open('matches.pickle', 'rb') as f:
    matches = pickle.loads(f.read())

In [4]:
len(matches)

39375

In [5]:
matches[0]

{'member_id': 113,
 'request_id': 28552,
 'mismatch': False,
 'auto_generated': True}

In [6]:
matches[-1]

{'member_id': 'Andrei',
 'request_id': 'Codiax',
 'mismatch': True,
 'auto_generated': False,
 'message': 'remove this from the matches list'}

In [7]:
matches = matches[:-1]

In [27]:
y = [] # the correct labels
baseline_predictions = []

# write code here...


# .. so that the asserts pass
assert len(y) == len(baseline_predictions) == len(matches)
assert sum(y) == 15739
assert sum(baseline_predictions) == 37010

In [8]:
y = [] # the correct labels
baseline_predictions = []

# write code here...
for match in matches:
    if match['auto_generated']:
        baseline_predictions.append(1)
        if match['mismatch']:
            y.append(0)
        else:
            y.append(1)
    else:
        baseline_predictions.append(0)
        if match['mismatch']:
            raise Exception('this is not possible', match)
        else:
            y.append(1)

# .. so that the asserts pass
assert len(y) == len(baseline_predictions) == len(matches)
assert sum(y) == 15739
assert sum(baseline_predictions) == 37010

In [9]:
import numpy as np

y = np.array(y)
baseline_predictions = np.array(baseline_predictions)

sum(baseline_predictions == y) / len(y)

0.3396911667597907

# Word2vec

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/word2vec.png" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>


<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/word2vec2.svg" width="1300vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/bert2.jpg" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/bert.png" width="1000vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>


## Install

```bash
pip install bert-serving-server  # server
pip install bert-serving-client  # client, independent of `bert-serving-server`
```

In [32]:
from bert_serving.client import BertClient
bc = BertClient()
results = bc.encode(['Lion is the king of the jungle',
    'The tiger hunts in this forest',
    'Everybody loves New York'])
results.shape

(3, 768)

In [33]:
results[0]

array([-2.99697369e-01,  1.81203470e-01, -7.88563862e-02, -3.74043509e-02,
        2.93722898e-01, -1.73312157e-01,  5.11504054e-01,  7.68373311e-01,
       -9.06428754e-01, -9.55983847e-02,  1.65843904e-01, -1.93865895e-01,
        6.25361353e-02,  1.34081438e-01, -5.44078588e-01,  1.73536643e-01,
       -1.31333902e-01,  3.50187451e-01,  5.98205440e-03,  8.96382406e-02,
       -2.00108364e-01, -2.36445114e-01, -3.51080060e-01, -2.88707376e-01,
        4.70682412e-01,  4.76228714e-01, -2.08968651e-02,  7.47544348e-01,
        1.91621125e-01,  2.32562140e-01,  4.80217002e-02,  5.57126880e-01,
        1.64404139e-01,  2.21324056e-01, -7.78836787e-01, -4.45689350e-01,
       -7.08792880e-02, -1.73808202e-01, -2.03022033e-01,  8.56973946e-01,
       -1.80435032e-01, -3.09874237e-01,  5.00707865e-01, -3.94362986e-01,
       -6.66409582e-02, -5.92953563e-01, -2.22736910e-01, -4.05069500e-01,
        4.83918041e-01, -3.81195098e-01, -2.34925807e-01,  1.20727114e-01,
       -1.08387136e+00, -

In [34]:
print(np.mean(results[0] * results[1]))
print(np.mean(results[0] * results[2]))
print(np.mean(results[1] * results[2]))

0.18777914
0.13391979
0.12317724


In [36]:
import pickle

with open('encodings.pickle', 'rb') as f:
  member_encodings, request_encodings = pickle.loads(f.read())

In [ ]:
X = []
for match in matches:
    X.append(member_encodings[match['member_id']] * \
        request_encodings[match['request_id']])

X = np.array(X)


X_train, X_test, y_train, y_test, _, bp_test = \
    train_test_split(X, y, baseline_pred, test_size=1000)


In [9]:
import numpy as np

y = np.array(y)
baseline_predictions = np.array(baseline_predictions)

sum(baseline_predictions == y) / len(y)

0.3396911667597907

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/acc.png" width="700vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

is this a good score or a bad score?

if doing pedestrian detection for autonomous driving, probably a bad score

but if you search for something on google and 3 of the first 10 results are what you were looking for, then it's probably a good score

in our case, we can ask for feedback from the moderators, since they're the ones that see these results

apart from that, for us this is just a number out of context

speaking of context, does anybody feel we might be missing something here?

this formula for accuracy is the general one; for binary classification we have the equivalent, but slightly more detailed:

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/acc2.png" width="700vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/confusion-matrix.png" width="800vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

did anybody figure out where i'm going with this?

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/acc2.png" width="700vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/confusion-matrix2.png" width="800vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

so it seems like we have no true negatives in our dataset

let's say we have 100 members and the algorithm only returns 5 matches; let's say the moderator adds another 5 members to the results; the remaining 90 members are the true negatives; both the algorithm and the human moderator haven't selected them; but they are not in our dataset

we need to add true negative to our dataset:
- for a better understanding of our algorithm's performance
- for training a ML model



In [ ]:
EXTEND_COUNT = 40000
# list of dictionaries {'member_id': int, 'request_id': int}
true_negatives = []

# generate 40000 random new matches ...


# ... such that these tests pass
assert len(true_negatives) == EXTEND_COUNT
true_negatives_set = {(tn['member_id'], tn['request_id']) for tn in true_negatives}
assert true_negatives_set == EXTEND_COUNT

old_matches_set = {(d['member_id'], d['request_id']) for d in matches}
member_ids_set = {{d['member_id'] for d in matches}}
request_ids_set = {{d['request_id'] for d in matches}}

assert len(true_negatives_set.difference(old_matches_set)) == EXTEND_COUNT
for tn in true_negatives:
    assert tn['member_id'] in member_ids_set
    assert tn['request_id'] in request_ids_set

In [16]:
EXTEND_COUNT = 40000
# list of dictionaries {'member_id': int, 'request_id': int}
true_negatives = []

# generate 40000 random new matches ...
import random

member_ids = list({d['member_id'] for d in matches})
request_ids = list({d['request_id'] for d in matches})
all_matches_set = {(d['member_id'], d['request_id']) for d in matches}

while True:
  member_id = member_ids[random.randint(0, len(member_ids) - 1)]
  request_id = request_ids[random.randint(0, len(request_ids) - 1)]
  if (member_id, request_id) in all_matches_set:
    continue
  true_negatives.append({
      'member_id': member_id,
      'request_id': request_id,
  })
  all_matches_set.add((member_id, request_id))

  if len(true_negatives) == EXTEND_COUNT:
    break

# ... such that these tests pass
assert len(true_negatives) == EXTEND_COUNT
true_negatives_set = {(tn['member_id'], tn['request_id']) for tn in true_negatives}
assert len(true_negatives_set) == EXTEND_COUNT

old_matches_set = {(d['member_id'], d['request_id']) for d in matches}
member_ids_set = {d['member_id'] for d in matches}
request_ids_set = {d['request_id'] for d in matches}

assert len(true_negatives_set.difference(old_matches_set)) == EXTEND_COUNT
for tn in true_negatives:
    assert tn['member_id'] in member_ids_set
    assert tn['request_id'] in request_ids_set

In [13]:
extended_matches = matches + true_negatives

# update y and baseline_predictions with the new true negatives...


# .. so that the asserts pass
assert len(y) == len(baseline_predictions) == len(extended_matches)
assert sum(y) == 15739
assert sum(baseline_predictions) == 37010

40000

In [17]:
extended_matches = matches + true_negatives

# update y and baseline_predictions with the new true negatives...
y = np.append(y, [0] * EXTEND_COUNT)
baseline_predictions = np.append(baseline_predictions, [0] * EXTEND_COUNT)

# .. so that the asserts pass
assert len(y) == len(baseline_predictions) == len(extended_matches)
assert sum(y) == 15739
assert sum(baseline_predictions) == 37010